In [ ]:
# Automatically reload modules when they are modified
%reload_ext autoreload
%autoreload 2

# Display plots and interactive visualizations inline (or in a separate window)
%matplotlib inline

In [ ]:
import pandas as pd 
import os
import geopandas as gpd 
import zipfile 
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import math
from shapely.geometry import Point, Polygon
import seaborn as sns
import folium
import pickle 
import numpy as np 
import warnings
import xarray as xr
import numpy as np
import json
import asyncio
import logging
import os
from concurrent.futures import ThreadPoolExecutor
from pathlib import Path
from typing import Any

import requests
from requests import Session
pd.set_option('display.max_columns', 80)
warnings.filterwarnings("ignore")
pd.options.display.float_format = '{:.3f}'.format

In [ ]:
# Set default style for plots
plt.style.use('ggplot')
sns.set_context('notebook', font_scale=1.2)

In [ ]:
def look_up_wijk(df_2023_db, df_20XX, WIJK_2023 = "WK_CODE_23", WIJK_20XX = "WK_CODE_20", plot = True):
    # df_reference: data 2023
    # df_reference: data 2023
    
    df_2023 = df_2023_db.copy()
    if 'area_complete' not in df_20XX:
        df_20XX['area_complete'] = df_20XX.geometry.area 
    if 'area_complete' not in df_2023:
        df_2023['area_complete'] = df_2023.geometry.area 
    intersection = gpd.overlay(df_2023, df_20XX , how="intersection")
    intersection['area_intersection'] = intersection.geometry.area

    intersection_20 = intersection.merge(df_20XX[[WIJK_20XX, "area_complete"]], on = WIJK_20XX, how = 'left')
    # print(intersection_20.head())
    intersection_20["area_prop"] = intersection_20["area_intersection"]/intersection_20["area_complete"]
    # intersection_20.tail()

    intersection_23 = intersection.merge(df_2023[[WIJK_2023, "area_complete"]], on = WIJK_2023, how = 'left')
    intersection_23["area_prop"] = intersection_23["area_intersection"]/intersection_23["area_complete"]
    # intersection_23.tail()

    cut_off = 0.90
    fully_inter = intersection_23[(intersection_23.area_prop > cut_off)]
    print(f"Not_fully_intersect: {df_2023[WIJK_2023].nunique() - fully_inter[WIJK_2023].nunique()}")

    not_fully = intersection_23[~intersection_23[WIJK_2023].isin(fully_inter[WIJK_2023])]
    not_fully = intersection_23[(intersection_23.area_prop <= cut_off) & (intersection_23.area_prop >= (1-cut_off))]

    tab_lookup_20_23 = pd.concat(
        [fully_inter[[WIJK_2023, WIJK_20XX, "area_prop"]], not_fully[[WIJK_2023, WIJK_20XX, "area_prop"]]]) 

    print(sum(df_2023[WIJK_2023].isin(tab_lookup_20_23[WIJK_2023])))
    print(df_2023[WIJK_2023].nunique())
    
    if (plot):
        ax = df_2023.plot(figsize=(10, 10), edgecolor='k', linewidth=0.5, legend=True, color = "red")
        df_2023[df_2023[WIJK_2023].isin(tab_lookup_20_23[WIJK_2023])].plot(ax = ax, figsize=(10, 10), edgecolor='k', linewidth=0.5, legend=True, color = 'blue')
        df_2023[~df_2023[WIJK_2023].isin(tab_lookup_20_23[WIJK_2023])].plot(ax = ax, figsize=(10, 10), edgecolor='k', linewidth=0.5, legend=True, color = 'green')
        # id_wijken_21.plot(ax = ax, column = 'LIHE', figsize=(10, 10), edgecolor='k', linewidth=0.5, legend=True, color = 'blue')
        # Set the title and display the plot
        plt.title(f"Plot of NL - Wijk", fontsize=15)
        plt.xlabel("Longitude")
        plt.ylabel("Latitude")
        plt.grid(True)
        plt.show() 
    return tab_lookup_20_23



In [ ]:
def weighted_average(df_2023, value, weight):
    val = df_2023[value]
    wt = df_2023[weight]
    return (val * wt).sum() / wt.sum()

In [ ]:
## Function to fetch data from KNMI 
logging.basicConfig()
logger = logging.getLogger(__name__)
logger.setLevel(os.environ.get("LOG_LEVEL", logging.INFO))


def download_dataset_file(
    session: Session,
    base_url: str,
    dataset_name: str,
    dataset_version: str,
    filename: str,
    directory: str,
    overwrite: bool,
) -> tuple[bool, str]:
    # if a file from this dataset already exists, skip downloading it.
    file_path = Path(directory, filename).resolve()
    if not overwrite and file_path.exists():
        logger.info(f"Dataset file '{filename}' was already downloaded.")
        return True, filename

    endpoint = f"{base_url}/datasets/{dataset_name}/versions/{dataset_version}/files/{filename}/url"
    get_file_response = session.get(endpoint)

    # retrieve download URL for dataset file
    if get_file_response.status_code != 200:
        logger.warning(f"Unable to get file: {filename}")
        logger.warning(get_file_response.content)
        return False, filename

    # use download URL to GET dataset file. We don't need to set the 'Authorization' header,
    # The presigned download URL already has permissions to GET the file contents
    download_url = get_file_response.json().get("temporaryDownloadUrl")
    return download_file_from_temporary_download_url(download_url, directory, filename)


def download_file_from_temporary_download_url(download_url, directory, filename):
    try:
        with requests.get(download_url, stream=True) as r:
            r.raise_for_status()
            with open(f"{directory}/{filename}", "wb") as f:
                for chunk in r.iter_content(chunk_size=8192):
                    f.write(chunk)
    except Exception:
        logger.exception("Unable to download file using download URL")
        return False, filename

    logger.info(f"Downloaded dataset file '{filename}'")
    return True, filename


def list_dataset_files(
    session: Session,
    base_url: str,
    dataset_name: str,
    dataset_version: str,
    params: dict[str, str],
) -> tuple[list[str], dict[str, Any]]:
    logger.info(f"Retrieve dataset files with query params: {params}")

    list_files_endpoint = f"{base_url}/datasets/{dataset_name}/versions/{dataset_version}/files"
    list_files_response = session.get(list_files_endpoint, params=params)

    if list_files_response.status_code != 200:
        raise Exception("Unable to list initial dataset files")

    try:
        list_files_response_json = list_files_response.json()
        dataset_files = list_files_response_json.get("files")
        dataset_filenames = list(map(lambda x: x.get("filename"), dataset_files))
        return dataset_filenames, list_files_response_json
    except Exception as e:
        logger.exception(e)
        raise Exception(e)


def get_max_worker_count(filesizes):
    size_for_threading = 10_000_000  # 10 MB
    average = sum(filesizes) / len(filesizes)
    # to prevent downloading multiple half files in case of a network failure with big files
    if average > size_for_threading:
        threads = 1
    else:
        threads = 10
    return threads


async def main():
    api_key = "<API_KEY>"
    dataset_name = "EV24"
    dataset_version = "2"
    base_url = "https://api.dataplatform.knmi.nl/open-data/v1"
    # When set to True, if a file with the same name exists the output is written over the file.
    # To prevent unnecessary bandwidth usage, leave it set to False.
    overwrite = False

    download_directory = "./dataset-download"

    # Make sure to send the API key with every HTTP request
    session = requests.Session()
    session.headers.update({"Authorization": api_key})

    # Verify that the download directory exists
    if not Path(download_directory).is_dir() or not Path(download_directory).exists():
        raise Exception(f"Invalid or non-existing directory: {download_directory}")

    filenames = []
    max_keys = 500
    next_page_token = None
    file_sizes = []
    # Use the API to get a list of all dataset filenames
    while True:
        # Retrieve dataset files after given filename
        dataset_filenames, response_json = list_dataset_files(
            session,
            base_url,
            dataset_name,
            dataset_version,
            {"maxKeys": f"{max_keys}", "nextPageToken": next_page_token},
        )
        file_sizes.extend(file["size"] for file in response_json.get("files"))
        # Store filenames
        filenames += dataset_filenames

        # If the result is not truncated, we retrieved all filenames
        next_page_token = response_json.get("nextPageToken")
        if not next_page_token:
            logger.info("Retrieved names of all dataset files")
            break

    logger.info(f"Number of files to download: {len(filenames)}")

    worker_count = get_max_worker_count(file_sizes)
    loop = asyncio.get_event_loop()

    # Allow up to 10 separate threads to download dataset files concurrently
    executor = ThreadPoolExecutor(max_workers=worker_count)
    futures = []

    # Create tasks that download the dataset files
    for dataset_filename in filenames:
        # Create future for dataset file
        future = loop.run_in_executor(
            executor,
            download_dataset_file,
            session,
            base_url,
            dataset_name,
            dataset_version,
            dataset_filename,
            download_directory,
            overwrite,
        )
        futures.append(future)

    # # Wait for all tasks to complete and gather the results
    future_results = await asyncio.gather(*futures)
    logger.info(f"Finished '{dataset_name}' dataset download")

    failed_downloads = list(filter(lambda x: not x[0], future_results))

    if len(failed_downloads) > 0:
        logger.warning("Failed to download the following dataset files:")
        logger.warning(list(map(lambda x: x[1], failed_downloads)))


In [ ]:
os.chdir("C:/Users/tejed002/OneDrive - Wageningen University & Research/PhD-WUR/01_WUR-Research/05_Papers/02_Paper_energy_poverty/analysis/data/wijken")

In [ ]:
### files and folder paths 
energy_cost = "energy_cost/Energiekosten - 2021 - Wijken (2023).csv"
energy_labels = "energy_labels/Energielabels (geldige labels Rijksoverheid) - Wijken (2023)_2020_2024.csv"
fragility_elderly = "fragility_elderly/50090NED_UntypedDataSet_18112024_154023.csv"
heating_system = "heating system/Hoofdverwarmingsinstallaties_woningen_2022_2023.xlsx"
infrastructure = "infrastructure/KEA_GroenGrijsPerBuurt_2023_03.xlsx"
renewable_energy_homes = "renewable_energy_homes/Hernieuwbare energie-installaties - Wijken (2023).csv"
energetic_efficiency = "climate_monitor/Inhoudelijke datasets per sector en thema - 2021 - Wijken.csv"

In [ ]:
# Wijken level dataset for 2023 
shp_path = '../../../Data/Raw/ShapeFiles/00_Processed'

with open(os.path.join(shp_path, 'dict_gdf.pkl'), 'rb') as file:
    dict_gdf = pickle.load(file)
    
with open(os.path.join(shp_path, 'dict_wijk_gdf.pkl'), 'rb') as file:
    dict_wijk_gdf = pickle.load(file)
    
with open(os.path.join(shp_path, 'dict_gemeente_gdf.pkl'), 'rb') as file:
    dict_gemeente_gdf = pickle.load(file)

In [ ]:
years = [2019, 2020, 2021, 2022, 2023]
for year in years:
    gdf_1 = dict_gdf[str(year)]
    gdf_1 = gdf_1[gdf_1["H2O"] == "NEE"]
    dict_gdf[str(year)] = gdf_1
    
    gdf_2 = dict_wijk_gdf[str(year)]
    gdf_2 = gdf_2[gdf_2["H2O"] == "NEE"]
    dict_wijk_gdf[str(year)] = gdf_2
    
    gdf_3 = dict_gemeente_gdf[str(year)]
    gdf_3 = gdf_3[gdf_3["H2O"] == "NEE"]
    dict_gemeente_gdf[str(year)] = gdf_3

In [ ]:
gdf_buurt = [f"gdf_buurt_{ii}" for ii in years]
gdf_wijk = [f"gdf_wijk_{ii}" for ii in years]
gdf_gemeente = [f"gdf_gemeente_{ii}" for ii in years]

In [ ]:
for name, value in zip(gdf_buurt, dict_gdf.values()):
    globals()[name] = value

In [ ]:
for name, value in zip(gdf_wijk, dict_wijk_gdf.values()):
    globals()[name] = value

In [ ]:
for name, value in zip(gdf_gemeente, dict_gemeente_gdf.values()):
    globals()[name] = value

# Calculate weights according area and population density 

In [ ]:
id_wijken_20 = gdf_wijk_2020[["WK_CODE", "GM_NAAM", "geometry", "AANT_INW"]]
id_wijken_20 = id_wijken_20.rename(columns = {"WK_CODE":"WK_CODE_20", "GM_NAAM":"GM_NAAM_20", "AANT_INW": "AANT_INW_20"})

id_wijken_21 = gdf_wijk_2021[["WK_CODE", "GM_NAAM", "geometry", "AANT_INW"]]
id_wijken_21 = id_wijken_21.rename(columns = {"WK_CODE":"WK_CODE_21", "GM_NAAM":"GM_NAAM_21", "AANT_INW": "AANT_INW_21"})

id_wijken_22 = gdf_wijk_2022[["WK_CODE", "GM_NAAM", "geometry", "AANT_INW"]]
id_wijken_22 = id_wijken_22.rename(columns = {"WK_CODE":"WK_CODE_22", "GM_NAAM":"GM_NAAM_22", "AANT_INW": "AANT_INW_22"})

id_wijken_23 = gdf_wijk_2023[["WK_CODE", "GM_NAAM", "GM_CODE", "geometry", "AANT_INW"]]
id_wijken_23 = id_wijken_23.rename(columns = {"WK_CODE":"WK_CODE_23", "GM_CODE":"GM_CODE_23", "GM_NAAM":"GM_NAAM_23", "AANT_INW": "AANT_INW_23"})

In [ ]:
create_weights = True
if create_weights:
    look_up_23_20 = look_up_wijk(id_wijken_23, df_20XX = id_wijken_20, WIJK_2023 = "WK_CODE_23", WIJK_20XX = "WK_CODE_20", plot = False)
    look_up_23_21 = look_up_wijk(id_wijken_23, id_wijken_21, WIJK_2023 = "WK_CODE_23", WIJK_20XX = "WK_CODE_21", plot = False)
    look_up_23_22 = look_up_wijk(id_wijken_23, id_wijken_22, WIJK_2023 = "WK_CODE_23", WIJK_20XX = "WK_CODE_22", plot = False)
    look_up_23_22 = look_up_wijk(id_wijken_23, id_wijken_22, WIJK_2023 = "WK_CODE_23", WIJK_20XX = "WK_CODE_22", plot = False)
    dict_look_up_23 = {"w_20":look_up_23_20, 
                       "w_21":look_up_23_21, 
                       "w_22":look_up_23_22
                       }
    with open("weights_homologate/dict_look_up_23.pkl", "wb") as file:
        pickle.dump(dict_look_up_23, file)
else:         
    with open("weights_homologate/dict_look_up_23.pkl", 'rb') as file:
        dict_look_up_23 = pickle.load(file)

# Data gathering and consolidation per topic

## Old frailty
**Definition**. The vulnerable group of over-65s in poor health thus ties in with three heat-vulnerable groups: the elderly, people living in social isolation, and people in weak health.

**Source**: https://www.klimaateffectatlas.nl/en/social-vulnerability-to-heat

In [ ]:
elderly_path = os.path.join(os.getcwd(), "fragility_elderly", '50090NED_UntypedDataSet_18112024_154023.csv')
dat_elderly = pd.read_csv(elderly_path, sep = ';')
dat_elderly.head()
elderly_cols = {"WijkenEnBuurten":"WK_CODE_20", 
                "BrozeGezondheid_45":"p_fragile_health_65"}
dat_elderly = dat_elderly.rename(columns=elderly_cols)

In [ ]:
dat_elderly["wijk_id"] = dat_elderly["WK_CODE_20"].str.contains('WK')
dat_elderly = dat_elderly[(dat_elderly["wijk_id"] == 1) & (dat_elderly["Marges"] == "MW00000")]
dat_elderly["WK_CODE_20"] = dat_elderly["WK_CODE_20"].str.strip()
dat_elderly.WK_CODE_20.nunique()

In [ ]:
dat_elderly.WK_CODE_20.duplicated().sum()

In [ ]:
# sum(~dat_elderly.WK_CODE_20.isin(look_up_23_20_ext.WK_CODE_20))

In [ ]:
look_up_23_20_ext = look_up_23_20.copy()
print(look_up_23_20_ext.shape)
look_up_23_20_ext = look_up_23_20_ext.merge(dat_elderly[["WK_CODE_20", "p_fragile_health_65"]], how = "left", on = "WK_CODE_20")
print(look_up_23_20_ext.shape)
look_up_23_20_ext.tail()

In [ ]:
look_up_23_20_ext["p_fragile_health_65"] = look_up_23_20_ext["p_fragile_health_65"].str.strip()
look_up_23_20_ext["p_fragile_health_65"] = np.where(look_up_23_20_ext["p_fragile_health_65"] == ".", np.NAN, look_up_23_20_ext["p_fragile_health_65"])

In [ ]:
look_up_23_20_ext["p_fragile_health_65"] = pd.to_numeric(look_up_23_20_ext["p_fragile_health_65"])
look_up_23_20_ext.dtypes

In [ ]:
# dat_elderly_g23 
def weighted_average(df_2023, value, weight):
   val = df_2023[value]
   wt = df_2023[weight]
   return (val * wt).sum() / wt.sum()

dat_elderly_g23 = look_up_23_20_ext.groupby("WK_CODE_23").apply(lambda group: weighted_average(group, "p_fragile_health_65", "area_prop")).reset_index(name = "p_fragile_health_65")

In [ ]:
dat_elderly_g23.head()

## Unemployment

Using unemployment rate from Statline at the neighborhood level: https://opendata.cbs.nl/portal.html?_la=nl&_catalog=CBS&tableId=85485NED&_theme=790 

net_labour_participation = working_population/. it is called Employment-to-Population Ratio

In [ ]:
unemploy_path = os.path.join(os.getcwd(), "unemployment", 'labour_participation_2021.csv')
dat_unemploy = pd.read_csv(unemploy_path, sep = ';')
dat_unemploy = dat_unemploy[dat_unemploy["Leeftijd"] == 52052] #"15 tot 75 jaar";""
dat_unemploy = dat_unemploy[dat_unemploy["WijkenEnBuurten"].str[:2] == "WK"] 
dat_unemploy.shape

In [ ]:
var_unemploy_1 = ["WijkenEnBuurten", "BeroepsEnNietBeroepsbevolking_1", "WerkzameBeroepsbevolking_2", "NettoArbeidsparticipatie_3", "Werknemer_4"]
var_unemploy_2 = ["work_not_population", "working_population", "net_labour_participation", "employed"]
unemploy_var = {
    "WijkenEnBuurten":"WK_CODE", 
    "BeroepsEnNietBeroepsbevolking_1": "work_not_population", 
    "WerkzameBeroepsbevolking_2": "working_population", 
    "NettoArbeidsparticipatie_3": "net_labour_participation",
    "Werknemer_4": "employed"
}
dat_unemploy = dat_unemploy[var_unemploy_1]
dat_unemploy = dat_unemploy.rename(columns=unemploy_var)

# dat_unemploy["unemployment_rate_GM"] = np.where(dat_unemploy["unemployment_rate_GM"] == '.', np.NAN , dat_unemploy["unemployment_rate_GM"])

# dat_unemploy["working_population"] = dat_unemploy["working_population"].str.strip()
# dat_unemploy["working_population"] = np.where(dat_unemploy["working_population"] == '.', np.NAN , dat_unemploy["working_population"])

In [ ]:
def dat_str_to_float(column):
    column = column.strip()
    # print(column)
    column = np.where(column == ".", np.NAN, column)
    column = column.astype(float)
    return column

In [ ]:
dat_unemploy[["net_labour_participation", "employed"]] = dat_unemploy[["net_labour_participation", "employed"]].applymap(lambda x: dat_str_to_float(x))

In [ ]:
dat_unemploy["net_labour_participation"] = dat_unemploy["net_labour_participation"]/100
dat_unemploy["employed"] = dat_unemploy["employed"]/100

In [ ]:
dat_unemploy["WK_CODE"] = dat_unemploy["WK_CODE"].str.strip()

## Buurtkaart 2021: Young children, Retired, Disability or permanent ill , Unemployed, Part-time employment, Housing tenure and market conditions

In [ ]:
gdf_wijk_2021[["P_KOOPWON", "P_HUURWON", "P_HUURCORP", "P_HUUROVVH", "P_HUKO_ONB"]]

In [ ]:

buurt_kaart_var = [
    "WK_CODE", 
    "GM_NAAM", 
    "GM_CODE", 
    "geometry", 
    "AANT_INW",     # Number of inhabitants [number]
    "WONINGEN",
    "P_00_14_JR", 
    "P_15_24_JR", 
    "P_25_44_JR", 
    "P_45_64_JR", 
    "P_65_EO_JR",   # Persons aged 65 and over [%]
    "P_HH_Z_K",     # households without children 
    "P_HH_M_K",     # households with children 
    "GEM_HH_GR",    # Average household size [number]
    # "P_ACTIEF",     # economically active actieven 15-75 jaar [%]
    "A_SOZ_OW",     # Reliant on state pension
    "WW_UIT_TOT",   # Unemployment 
    "AO_UIT_TOT",   # receive a disability benefit under several circumstances  
    "A_WMO_T",      # Precarious part time
    "P_KOOPWON",    # Owner-occupied homes percentage 
    "P_HUURWON",    # Rental properties total percentage 
    "P_HUURCORP",   # Rental properties owned by housing corporations  
    "G_GAS_HU",     # Gas consumption m3 - Rental
    "G_GAS_KO",     # Gas consumption m3 - Owner Occupied
    "G_ELE_HU",    # Electricity consumption kWh - Rental
    "G_ELE_KO",     # Electricity consumption kWh - Owner Occupied
    "G_GAS_TOT",    # Gas consumption m3 kWh - Total
    "G_ELEK_TOT",     # Electricity consumption kWh  - total
]
np.isin(buurt_kaart_var, gdf_wijk_2021.columns)
buurt_kaart_cols = {    
    "WK_CODE":"WK_CODE_21", 
    "GM_NAAM":"GM_NAAM_21", 
    "AANT_INW": "AANT_INW_21",              # Number of inhabitants [number]  
    "WONINGEN":"housing_stock",
    "P_00_14_JR": "p_0_14",
    "P_15_24_JR": "p_15_24",
    "P_25_44_JR": "p_25_44",
    "P_45_64_JR": "p_45_64",   
    "P_65_EO_JR": "p_65_older",             # Persons aged 65 and over [%]    
    "NUMBER_HH": "total_households",        # Total households
    "P_HH_Z_K":"p_without_young_children",  # households without children 
    "P_HH_M_K":"p_with_young_children",     # households with children 
    "GEM_HH_GR" : "t_average_household_size",
    # "P_ACTIEF" : "t_economic_active_pop", 
    "A_SOZ_OW": "t_pension",                # Reliant on state pension
    "AO_UIT_TOT": "t_disability",           # receive a disability benefit under several circumstances  
    "WW_UIT_TOT":"t_unemployment",          # Unemployment 
    "A_WMO_T":"t_precarious_part_time",     # Precarious part time
    "P_KOOPWON":"p_owner_ocupied_home",     # Owner-occupied homes
    "P_HUURCORP":"p_rental_corp_home",      # Rental properties total
    "P_HUURWON":"p_rental_home",       # Rental properties owned by housing corporations
    "G_GAS_HU":"t_gas_consum_rental",       # Gas consumption m3 - Rental
    "G_GAS_KO":"t_gas_consum_owner",        # Gas consumption m3 - Owner Occupied
    "G_ELE_HU":"t_elec_consum_rental",      # Electricity consumption kWh - Rental
    "G_ELE_KO":"t_elec_consum_owner",       # Electricity consumption kWh - Owner Occupied
    "G_GAS_TOT":"t_gas_consum_total",       # Gas consumption m3 kWh - Total
    "G_ELEK_TOT":"t_elec_consum_total"      # Electricity consumption kWh  - total   
    }

In [ ]:
id_wijken_21 = gdf_wijk_2021[["WK_CODE", "GM_CODE","GM_NAAM", "geometry", "AANT_INW"]]
id_wijken_21 = id_wijken_21.rename(columns = {"WK_CODE":"WK_CODE_21", "GM_CODE":"GM_CODE_21", "GM_NAAM":"GM_NAAM_21", "AANT_INW": "AANT_INW_21"})

In [ ]:
dat_vulnerability_buurtkaart  = gdf_wijk_2021[buurt_kaart_var]
dat_vulnerability_buurtkaart = dat_vulnerability_buurtkaart.rename(columns=buurt_kaart_cols)
# dat_vulnerability_buurtkaart.head()
    

In [ ]:
columns = dat_vulnerability_buurtkaart.columns
for ii in range(4,len(dat_vulnerability_buurtkaart.columns)):
    column = columns[ii]
    dat_vulnerability_buurtkaart[column] = np.where(dat_vulnerability_buurtkaart[column] < 0, np.nan, dat_vulnerability_buurtkaart[column])
    # print (dat_vulnerability_buurtkaart.columns[column])

In [ ]:
print(dat_vulnerability_buurtkaart.shape)
dat_vulnerability_buurtkaart = dat_vulnerability_buurtkaart.merge(dat_unemploy, left_on="WK_CODE_21", right_on="WK_CODE", how = "left")
print(dat_vulnerability_buurtkaart.shape)

In [ ]:
dat_vulnerability_buurtkaart["employed_population"] =  dat_vulnerability_buurtkaart["employed"] * dat_vulnerability_buurtkaart["working_population"]
dat_vulnerability_buurtkaart["p_unemployment_rate"] = dat_vulnerability_buurtkaart["t_unemployment"]/(dat_vulnerability_buurtkaart["t_unemployment"] + dat_vulnerability_buurtkaart["employed_population"]) * 100

In [ ]:
dat_vulnerability_buurtkaart["t_unemployment"].sum() / (dat_vulnerability_buurtkaart["t_unemployment"].sum() + dat_vulnerability_buurtkaart["employed_population"].sum())

In [ ]:
dat_vulnerability_buurtkaart["t_0_14"] = (dat_vulnerability_buurtkaart["p_0_14"]/100) * dat_vulnerability_buurtkaart["AANT_INW_21"]
dat_vulnerability_buurtkaart["t_15_24"] = (dat_vulnerability_buurtkaart["p_15_24"]/100) * dat_vulnerability_buurtkaart["AANT_INW_21"]
dat_vulnerability_buurtkaart["t_25_44"] = (dat_vulnerability_buurtkaart["p_25_44"]/100) * dat_vulnerability_buurtkaart["AANT_INW_21"]
dat_vulnerability_buurtkaart["t_45_64"] = (dat_vulnerability_buurtkaart["p_45_64"]/100) * dat_vulnerability_buurtkaart["AANT_INW_21"]
dat_vulnerability_buurtkaart["t_65_older"] = (dat_vulnerability_buurtkaart["p_65_older"]/100) * dat_vulnerability_buurtkaart["AANT_INW_21"]
dat_vulnerability_buurtkaart["t_15_older"] = (dat_vulnerability_buurtkaart["t_15_24"] + dat_vulnerability_buurtkaart["t_25_44"] +
                                              dat_vulnerability_buurtkaart["t_45_64"] + dat_vulnerability_buurtkaart["t_65_older"])

dat_vulnerability_buurtkaart["t_less_65"] = (dat_vulnerability_buurtkaart["t_0_14"] + dat_vulnerability_buurtkaart["t_15_24"] + 
                                             dat_vulnerability_buurtkaart["t_25_44"] + dat_vulnerability_buurtkaart["t_45_64"] )


In [ ]:
dat_vulnerability_buurtkaart["p_pension_coverage_rate"] = np.where(dat_vulnerability_buurtkaart["t_65_older"]  == 0, np.NAN, 
                                                     dat_vulnerability_buurtkaart["t_pension"]/dat_vulnerability_buurtkaart["t_65_older"] * 100)
dat_vulnerability_buurtkaart["p_pension_coverage_rate"] = np.where(dat_vulnerability_buurtkaart["p_pension_coverage_rate"]>100, 100, dat_vulnerability_buurtkaart["p_pension_coverage_rate"])

In [ ]:
dat_vulnerability_buurtkaart["p_precarious_part_time"] = dat_vulnerability_buurtkaart["t_precarious_part_time"] / dat_vulnerability_buurtkaart["t_15_older"] * 100

In [ ]:
dat_vulnerability_buurtkaart["p_disability"] = dat_vulnerability_buurtkaart["t_disability"] / dat_vulnerability_buurtkaart["t_less_65"] * 100

In [ ]:
dat_vulnerability_buurtkaart[["t_pension", "p_65_older", "p_pension_coverage_rate", "t_65_older", "p_precarious_part_time", "p_disability"]].describe()

In [ ]:
#### calculate totals according population and total households, calculate according totals and then calculate parcentages accordingly. 
# I need to include 2023 total population and households, 

In [ ]:
sel_cols_21 = ["WK_CODE_21", 'p_without_young_children', "p_with_young_children", 'p_pension_coverage_rate', 'p_unemployment_rate', 'p_precarious_part_time', "p_disability", 
               'p_owner_ocupied_home', 'p_rental_home', 't_gas_consum_rental', 't_gas_consum_owner', 't_elec_consum_rental', 't_elec_consum_owner', 't_gas_consum_total',
               't_elec_consum_total']

In [ ]:
look_up_23_21_ext = look_up_23_21.copy()
print(look_up_23_21_ext.shape)
look_up_23_21_ext = look_up_23_21_ext.merge(dat_vulnerability_buurtkaart[sel_cols_21], how = "left", on = "WK_CODE_21")
print(look_up_23_21_ext.shape)
look_up_23_21_ext.tail()

In [ ]:
columns_to_compute = sel_cols_21[1:]
# ['young_children', 'pension',
#        'unemployment', 'precarious_part_time', 'owner_ocupied_home',
#        'rental_home', 'gas_consum_rental', 'gas_consum_owner',
#        'elec_consum_rental', 'elec_consum_owner', 'gas_consum_total',
#        'elec_consum_total']
dat_buurt_kaart_g23 = (
    look_up_23_21_ext.groupby("WK_CODE_23")
    .apply(lambda group: pd.Series(
            {col: weighted_average(group, col, "area_prop") for col in columns_to_compute}))
    .reset_index()
)
dat_buurt_kaart_g23.head()

## Heating system

In [ ]:
dat_heating_sys = pd.read_excel(os.path.join(os.getcwd(), heating_system), sheet_name = "filter_wijk")

In [ ]:
heating_sys_cols = {    
    "Wijken en buurten":"WK_CODE_23", 
    "Individual CV":"p_individual_CV", 
    "Block heating":"p_block_heating", 
    "District heating with high gas consumption": "p_district_heat_high_gas_consum",
    "District heating with low gas consumption":"p_district_heat_low_gas_consum",
    "District heating without gas consumption": "p_district_heat_no_gas_consum", 
    "Electrically heated with high gas consumption":"p_elec_heat_high_gas_consum",
    "Electrically heated with low gas consumption":"p_elec_heat_low_gas_consum",
    "Electrically heated without gas consumption":"p_elec_heat_no_gas_consum",
    "Total without gas consumption (excl unknown)":"p_total_no_gas",
    "Type of installation unknown":"unknown"
    }

In [ ]:
dat_heating_sys = dat_heating_sys.rename(columns=heating_sys_cols)
dat_heating_sys.head()

In [ ]:
dat_heating_sys["p_district_heat"] = dat_heating_sys[["p_district_heat_high_gas_consum", "p_district_heat_low_gas_consum", "p_district_heat_no_gas_consum"]].sum(axis = 1)
dat_heating_sys["p_elec_heat"] = dat_heating_sys[["p_elec_heat_high_gas_consum", "p_elec_heat_low_gas_consum", "p_elec_heat_no_gas_consum"]].sum(axis = 1)
dat_heating_sys["p_total_homes"] = dat_heating_sys[["p_individual_CV", "p_block_heating", "p_district_heat", "p_elec_heat"]].sum(axis = 1)
print(dat_heating_sys["p_total_homes"].describe())
dat_heating_sys = dat_heating_sys[dat_heating_sys["Periode"] == 2022]
print(dat_heating_sys.shape)

In [ ]:
dat_heating_sys_g23 = dat_heating_sys[["WK_CODE_23", 
                                       "p_individual_CV", "p_block_heating", "p_district_heat", "p_elec_heat", 
                                       "p_district_heat_high_gas_consum", "p_district_heat_low_gas_consum", "p_district_heat_no_gas_consum", 
                                       "p_elec_heat_high_gas_consum", "p_elec_heat_low_gas_consum", "p_elec_heat_no_gas_consum"]]
dat_heating_sys_g23.head()

## Energetic efficiency 

In [ ]:
dat_energetic_efficiency = pd.read_csv(os.path.join(os.getcwd(), energetic_efficiency), header=0, delimiter=";")
dat_energetic_efficiency.head()

In [ ]:
list_split = []
for string in dat_energetic_efficiency["Gemeenten"]:
    list_split.append(string.rsplit(":", 1))

In [ ]:
df_split = pd.DataFrame(list_split)
df_split = df_split.rename(columns = {0:"GM_NAAM", 1:"WK_NAAM"})
sum(~df_split.GM_NAAM.isin(gdf_wijk_2022.GM_NAAM))
df_split.GM_NAAM[~df_split.GM_NAAM.isin(gdf_wijk_2022.GM_NAAM)].unique()

In [ ]:
# [string.str.contains(":") for string in df_split.GM_NAAM.str.contains(:)]
def reprocess_split(df_split):
    list_gnaam = []
    list_wknaam = []
    for ii in range(df_split.shape[0]):
        ll = df_split["GM_NAAM"].iloc[ii].rsplit(":", 1)
        list_gnaam.append(ll[0])
        list_wknaam.append(ll[1] +":"+df_split["WK_NAAM"].iloc[ii])
    # print(list_gnaam)    
    # print(list_wkcode)    
    df_split["new_gnaam"] = list_gnaam
    df_split["new_wknaam"] = list_wknaam
    return(df_split)
        # list_split.append(string.rsplit(":", 1))
df_split["id"] = range(df_split.shape[0])
df_split["to_process"] = df_split.GM_NAAM.str.contains(":")
dat_reprocess_string = reprocess_split(df_split[df_split.to_process]) 
    
# df_split[df_split.to_process]
# df_split[(~df_split.to_process) & (df_split.GM_NAAM == "Utrecht")]

In [ ]:
df_split = df_split.merge(dat_reprocess_string[["id", "new_gnaam", "new_wknaam"]], how = "left", on = "id")
df_split.head()

In [ ]:
new_gnaam = np.where(~df_split.new_gnaam.isna(), df_split.new_gnaam, df_split.GM_NAAM)
new_wkcode = np.where(~df_split.new_gnaam.isna(), df_split.new_wknaam, df_split.WK_NAAM)
df_split["GM_NAAM_23"] = new_gnaam
df_split["WK_NAAM_23"] = new_wkcode

In [ ]:
df_split[df_split.to_process]

In [ ]:
print(sum(~df_split.GM_NAAM_23.isin(gdf_wijk_2022["GM_NAAM"])))
df_split.GM_NAAM[~df_split.GM_NAAM_23.isin(gdf_wijk_2023["GM_NAAM"])].unique()
#### here issues 
# 'Voorne aan Zee': not found 
# 'Onbekend': unknown 

In [ ]:
df_split["GM_NAAM_23"] = np.where(df_split["GM_NAAM_23"] == 'Den Haag', "'s-Gravenhage", df_split["GM_NAAM_23"]) 
df_split.GM_NAAM_23[~df_split.GM_NAAM_23.isin(gdf_wijk_2023["GM_NAAM"])].unique()

In [ ]:
df_split["WK_NAAM_23"] = df_split["WK_NAAM_23"].apply(lambda x: x.decode('utf-8', errors='replace') if isinstance(x, bytes) else x)

In [ ]:
# gdf_wijk_2021[gdf_wijk_2021["WK_NAAM"].str.contains("Wijk 01: Houthem - Sint Gerlach")]
df_split["WK_NAAM_23"] = df_split["WK_NAAM_23"].str.strip()
df_split.WK_NAAM_23[~df_split.WK_NAAM_23.isin(gdf_wijk_2023["WK_NAAM"])].unique()

In [ ]:
gdf_wijk_2023["WK_NAAM"][~gdf_wijk_2023["WK_NAAM"].isin(df_split.WK_NAAM_23)].unique()

In [ ]:
mapping = {
    'Babyloni?�nbroek': 'Babyloniënbroek',
    'Chass?�buurt': 'Chassébuurt',
    'Castelr?�': 'Castelré',
    '1e Exlo?�rmond': '1e Exloërmond',
    '2e Exlo?�rmond': '2e Exloërmond',
    'Exlo?�rveen': 'Exloërveen',
    's-Gravenpolder': "'s-Gravenpolder",
    's-Heer Abtskerke': "'s-Heer Abtskerke",
    's-Heerenhoek': "'s-Heerenhoek",
    's Gravenmoer': "'s Gravenmoer",
    'Skarsterl?�n': 'Skarsterlân',
    's-Gravendeel': "'s-Gravendeel",
    't Veld': "'t Veld",
    't Goy': "'t Goy",
    't Goy Buitengebied': "'t Goy Buitengebied",
    't Heen': "'t Heen",
    'Nijl?�n & De Zwette': 'Nijlân & De Zwette',
    'Aldl?�n & De Hemrik': 'Aldlân & De Hemrik',
    't Lien / De Rietvink': "'t Lien / De Rietvink",
    'Wijk 03 Oh?� en Laak': 'Wijk 03 Ohé en Laak',
    "t Westend / 't Seuverick": "'t Westend / 't Seuverick",
    'Jonkersl?�n': 'Jonkerslân',
    'Mari?�nheem': 'Mariënheem',
    'Sint Odili?�nberg': 'Sint Odiliënberg',
    'Wijk 02 M??nein': 'Wijk 02 Mûnein',
    'Wijk 05 Earnew?�ld': 'Wijk 05 Earnewâld',
    'Wijk onbekend': 'Wijk onbekend'
}

df_split["WK_NAAM_23"] = df_split["WK_NAAM_23"].replace(mapping)

In [ ]:
print(df_split.WK_NAAM_23[~df_split.WK_NAAM_23.isin(gdf_wijk_2023["WK_NAAM"])].unique())
print(gdf_wijk_2023["WK_NAAM"][~gdf_wijk_2023["WK_NAAM"].isin(df_split.WK_NAAM_23)].unique())

In [ ]:
print(dat_energetic_efficiency.shape)
dat_energetic_efficiency = pd.concat([dat_energetic_efficiency, df_split], axis=1)
print(dat_energetic_efficiency.shape)

In [ ]:
print(dat_energetic_efficiency.shape)
dat_energetic_efficiency = dat_energetic_efficiency.merge(gdf_wijk_2023[["WK_CODE", "WK_NAAM", "GM_CODE", "GM_NAAM"]], left_on = ["GM_NAAM_23", "WK_NAAM_23"], right_on= ["GM_NAAM", "WK_NAAM"], how = "left")
print(dat_energetic_efficiency.shape)

In [ ]:
dat_energetic_efficiency_g23 = dat_energetic_efficiency.copy()

In [ ]:
dat_energetic_efficiency_g23.iloc[:,7:15]

In [ ]:
dat_energetic_efficiency_g23.columns

In [ ]:
energ_eff_cols = {
       'Percentage woningen met geregistreerde zonnepanelen|2021': "p_homes_with_solar_panels", ### Export again NaN 
       'Percentage woningen dat is voorzien van een geldig energielabel|2021': "p_homes_valid_energy_label",
       'Percentage gelabelde woningen met geldig energielabel A t/m A++++|2021': "p_homes_energy_label_A_Aplus",
       'Percentage gelabelde woningen met geldig energielabel B|2021': "p_homes_energy_label_B",
       'Percentage gelabelde woningen met geldig energielabel C|2021': "p_homes_energy_label_C",
       'Percentage gelabelde woningen met geldig energielabel D|2021': "p_homes_energy_label_D",
       'Percentage gelabelde woningen met geldig energielabel E|2021': "p_homes_energy_label_E",
       'Percentage gelabelde woningen met geldig energielabel F|2021': "p_homes_energy_label_F",
       'Percentage gelabelde woningen met geldig energielabel G|2021': "p_homes_energy_label_G",
       'Woningen met geregistreerd energielabel|2021' : "t_homes_register_energy_label",    
       'Woningdichtheid (aantal woningen per hectare)|2021': "housing_density", 
       'Huurwoningen in bezit van woningcorporaties|2021': "p_rental_own_hous_association", 
       'GM_CODE': 'GM_CODE_23',
       'WK_CODE': 'WK_CODE_23' 
}
dat_energetic_efficiency_g23 = dat_energetic_efficiency_g23.rename(columns=energ_eff_cols)
# cols_energetic_efficiency = ['WK_CODE_23', 'housing_density', 'p_rental_own_hous_association']
# dat_energetic_efficiency_g23[cols_energetic_efficiency].head()
### pending energy labels

In [ ]:
dat_energetic_efficiency_g23.columns

In [ ]:
def dat_str_to_float(column):
    import re
    # print(column)
    column = str(column)
    column = re.sub(r'[\?\-\,]', '.', column)
    column = np.where(column == ".", np.NAN, column)
    column = column.astype(float)
    return column
    

In [ ]:
object_to_float = [ 'p_homes_with_solar_panels',  'p_homes_energy_label_A_Aplus',  'p_homes_energy_label_B',  'p_homes_energy_label_C',  'p_homes_energy_label_D',  'p_homes_energy_label_F',
 'p_homes_energy_label_E',  'p_homes_energy_label_G',  'housing_density',  'p_rental_own_hous_association']
dat_energetic_efficiency_g23[object_to_float] = dat_energetic_efficiency_g23[object_to_float].applymap(lambda x: dat_str_to_float(x))

In [ ]:
# # dat_energetic_efficiency_g23["housing_density"] = dat_energetic_efficiency_g23["housing_density"].str.replace(',', '.').astype(float)
# dat_energetic_efficiency_g23["p_rental_own_hous_association"] = (
#     dat_energetic_efficiency_g23["p_rental_own_hous_association"]
#     .str.replace('?', '', regex=False)
#     .replace('', None)
#     .astype(float)
# )
# dat_energetic_efficiency_g23[cols_energetic_efficiency].head()

## Climatic and environmental variables

In [ ]:
### files and folder paths 
energy_cost = "energy_cost/Energiekosten - 2021 - Wijken (2023).csv"
energy_labels = "energy_labels/Energielabels (geldige labels Rijksoverheid) - Wijken (2023)_2020_2024.csv"
fragility_elderly = "fragility_elderly/50090NED_UntypedDataSet_18112024_154023.csv"
heating_system = "heating system/Hoofdverwarmingsinstallaties_woningen_2022_2023.xlsx"
infrastructure = "infrastructure/KEA_GroenGrijsPerBuurt_2023_03.xlsx"
renewable_energy_homes = "renewable_energy_homes/Hernieuwbare energie-installaties - Wijken (2023).csv"
energetic_efficiency = "climate_monitor/Inhoudelijke datasets per sector en thema - 2021 - Wijken.csv"
environmental_climatic = "climatic_environmental/KEA_GroenGrijsPerBuurt_2023_03.xlsx"

In [ ]:
dat_green_gray_infra = pd.read_excel(os.path.join(os.getcwd(), environmental_climatic), sheet_name = "KEA_GroenGrijsPerBuurt")
dat_green_gray_infra.head()

In [ ]:
gdf_buurt_2020.crs

In [ ]:
sum(~gdf_buurt_2020.apply(lambda x: x.geometry.is_valid, axis=1))

In [ ]:
gdf_buurt_2020['area_complete'] = gdf_buurt_2020.geometry.area

In [ ]:
gdf_wijk_2020['area_complete'] = gdf_wijk_2020.geometry.area

**PercentageBoom**:	Totaal percentage boom per buurt (boom, exclusief agrarisch)

**PercentageGrijs**: Totaal percentage grijs per buurt (totaal - groen - boom - water, exclusief agrarisch)

**PercentageGroen**: Totaal percentage groen per buurt (groen + boom, exclusief agrarisch)


In [ ]:
print(dat_green_gray_infra.shape)
dat_green_gray_infra = dat_green_gray_infra.merge(gdf_buurt_2020[["BU_CODE", "WK_CODE", "GM_CODE", "GM_NAAM", "area_complete"]], 
                                                  left_on = "buurtcode", right_on = "BU_CODE")
print(dat_green_gray_infra.shape)

In [ ]:
# # dat_elderly_g23 
# def weighted_average(df_2023, value, weight):
#    val = df_2023[value]
#    wt = df_2023[weight]
#    return (val * wt).sum() / wt.sum()

In [ ]:
# dat_green_gray_infra["p_green_buurt"] = (dat_green_gray_infra["PercentageGroen"]/100) * dat_green_gray_infra["area_complete"]
# dat_green_gray_infra["p_grey_buurt"] = (dat_green_gray_infra["PercentageGrijs"]/100) * dat_green_gray_infra["area_complete"]

In [ ]:
dat_green_gray_infra = (
    dat_green_gray_infra.groupby("WK_CODE")
    .apply(lambda group: pd.Series(
            {col: weighted_average(group, col, "area_complete") for col in ["PercentageGroen", "PercentageGrijs"]}))
    .reset_index()
)
dat_green_gray_infra.head()

In [ ]:
dat_green_gray_infra[["PercentageGroen", "PercentageGrijs"]].describe()

In [ ]:
# print(dat_green_gray_infra.shape)
# dat_green_gray_infra = dat_green_gray_infra.merge(gdf_wijk_2020[["WK_CODE", "area_complete"]],  on = "WK_CODE")
# print(dat_green_gray_infra.shape)

In [ ]:
# lll = dat_green_gray_infra[dat_green_gray_infra["total_green_buurt"] > dat_green_gray_infra["area_complete"]]
# gdf_wijk_2020[gdf_wijk_2020.WK_CODE.isin(lll.WK_CODE)]

In [ ]:
# dat_green_gray_infra["p_green_wijk"] = dat_green_gray_infra["total_green_buurt"] / dat_green_gray_infra["area_complete"] 
# dat_green_gray_infra["p_grey_wijk"] = dat_green_gray_infra["total_grey_buurt"] / dat_green_gray_infra["area_complete"] 

In [ ]:
# dat_green_gray_infra[dat_green_gray_infra.p_green_wijk > 1]

In [ ]:
dat_green_gray_infra = dat_green_gray_infra.rename(columns = {"PercentageGroen":"p_green", "PercentageGrijs":"p_gray"})

In [ ]:
print(look_up_23_20_ext.shape)
look_up_23_20_ext = look_up_23_20_ext.merge(dat_green_gray_infra[["WK_CODE", "p_green", "p_gray"]], how = "left", 
                                            left_on = "WK_CODE_20", right_on = "WK_CODE")
print(look_up_23_20_ext.shape)

In [ ]:
dat_green_gray_g23 = (
    look_up_23_20_ext.groupby("WK_CODE_23")
    .apply(lambda group: pd.Series(
            {col: weighted_average(group, col, "area_prop") for col in ["p_green", "p_gray"]}))
    .reset_index()
)
dat_green_gray_g23.head()

In [ ]:
dat_green_gray_g23.plot.scatter("p_green", "p_gray")
# plt.show()

## Inequality and GINI

In [ ]:
### files and folder paths 
energy_cost = "energy_cost/Energiekosten - 2021 - Wijken (2023).csv"
energy_labels = "energy_labels/Energielabels (geldige labels Rijksoverheid) - Wijken (2023)_2020_2024.csv"
fragility_elderly = "fragility_elderly/50090NED_UntypedDataSet_18112024_154023.csv"
heating_system = "heating system/Hoofdverwarmingsinstallaties_woningen_2022_2023.xlsx"
infrastructure = "infrastructure/KEA_GroenGrijsPerBuurt_2023_03.xlsx"
renewable_energy_homes = "renewable_energy_homes/Hernieuwbare energie-installaties - Wijken (2023).csv"
energetic_efficiency = "climate_monitor/Inhoudelijke datasets per sector en thema - 2021 - Wijken.csv"
environmental_climatic = "climatic_environmental/KEA_GroenGrijsPerBuurt_2023_03_Processed.xlsx"
inequality = "income_inequality/VSO Tabel Hoofdstuk 7 2021.xlsx"

In [ ]:
dat_gini_g23 = pd.read_excel(os.path.join(os.getcwd(), inequality), sheet_name = "GINI")
dat_gini_g23 = dat_gini_g23[dat_gini_g23["BU_CODE"] == "Totaal"]

In [ ]:
dat_gini_g23["WK_CODE_23"] = "WK" + dat_gini_g23["GM_CODE"] + dat_gini_g23["WK_CODE"]
dat_gini_g23.head()

In [ ]:
dat_gini_g23 = dat_gini_g23[["WK_CODE_23", "GINI"]]
dat_gini_g23.head()

## Climatic HDD and CDD 

In [ ]:
# # download HDD and CDD
# download_dataset_file(
# session,
# base_url,
# dataset_name, 
# dataset_version, 
# filename = 'jaarlijks-cdd-climatology-ANN-2020_gridded.nc', 
# directory = download_directory,
# overwrite = True
# )

In [ ]:
# download_dataset_file(
# session,
# base_url,
# dataset_name, 
# dataset_version, 
# filename = 'jaarlijks-hdd-climatology-ANN-2020_gridded.nc', 
# directory = download_directory,
# overwrite = True
# )

### CDD

In [ ]:
# Open the CDD file 
file_path = "./climatic_environmental/jaarlijks-cdd-climatology-ANN-2020_gridded.nc"
dat_cdd = xr.open_dataset(file_path)

In [ ]:
dat_cdd

In [ ]:
dat_cdd = dat_cdd.to_dataframe().reset_index()

In [ ]:
dat_cdd.interpolatedObs.isna().value_counts()

In [ ]:
dat_cdd = dat_cdd[~dat_cdd.interpolatedObs.isna()]
print(dat_cdd.shape)
dat_cdd.head()

In [ ]:
dat_cdd["x"] = pd.to_numeric(dat_cdd["x"], errors="coerce")
dat_cdd["y"] = pd.to_numeric(dat_cdd["y"], errors="coerce")

In [ ]:
dat_cdd = dat_cdd[["x", "y", "interpolatedObs"]]
dat_cdd_proj = dat_cdd.copy()

In [ ]:
from pyproj import Proj, transform

# Define the input CRS (from the Proj4 string)
input_crs = Proj("+proj=sterea +lat_0=52.15616055555555 +lon_0=5.38763888888889 "
                 "+k=0.9999079 +x_0=155000 +y_0=463000 +ellps=bessel "
                 "+towgs84=565.4171,50.3319,465.5524,-0.398957388243134,"
                 "0.343987817378283,-1.87740163998045,4.0725 +units=m +no_defs")
dat_cdd_proj = gpd.GeoDataFrame(dat_cdd_proj, geometry=gpd.points_from_xy(dat_cdd_proj["x"], dat_cdd_proj["y"]))

In [ ]:
dat_cdd_proj.set_crs(
            "+proj=sterea +lat_0=52.15616055555555 +lon_0=5.38763888888889 "
            "+k=0.9999079 +x_0=155000 +y_0=463000 +ellps=bessel "
            "+towgs84=565.4171,50.3319,465.5524,-0.398957388243134,"
            "0.343987817378283,-1.87740163998045,4.0725 +units=m +no_defs", 
            inplace=True)

In [ ]:
dat_cdd_proj = dat_cdd_proj.to_crs(epsg=28992)

In [ ]:
dat_cdd_proj = gpd.sjoin(dat_cdd_proj, id_wijken_23[["WK_CODE_23", "GM_CODE_23", "geometry"]], how = "left", predicate = "within")
dat_cdd_proj.head()

In [ ]:
print(sum(dat_cdd_proj.WK_CODE_23.isnull())/dat_cdd_proj.shape[0])
dat_cdd_proj = dat_cdd_proj[~dat_cdd_proj["WK_CODE_23"].isnull()]

In [ ]:
dat_cdd_g23 = (dat_cdd_proj.
               groupby(["WK_CODE_23", "GM_CODE_23"])["interpolatedObs"].
               mean().
               reset_index().
               rename(columns = {"interpolatedObs":"CDD"})
               )

### HDD

In [ ]:
# Open the CDD file 
file_path = "./climatic_environmental/jaarlijks-hdd-climatology-ANN-2020_gridded.nc"
dat_hdd = xr.open_dataset(file_path)

In [ ]:
dat_hdd

In [ ]:
dat_hdd = dat_hdd.to_dataframe().reset_index()

In [ ]:
dat_hdd.interpolatedObs.isna().value_counts()

In [ ]:
dat_hdd.describe()

In [ ]:
dat_hdd = dat_hdd[~dat_hdd.interpolatedObs.isna()]
print(dat_hdd.shape)
dat_hdd.head()

In [ ]:
dat_hdd["x"] = pd.to_numeric(dat_hdd["x"], errors="coerce")
dat_hdd["y"] = pd.to_numeric(dat_hdd["y"], errors="coerce")

In [ ]:
dat_hdd = dat_hdd[["x", "y", "interpolatedObs"]]
dat_hdd_proj = dat_hdd.copy()

In [ ]:
from pyproj import Proj, transform

# Define the input CRS (from the Proj4 string)
dat_hdd_proj = gpd.GeoDataFrame(dat_hdd_proj, geometry=gpd.points_from_xy(dat_hdd_proj["x"], dat_hdd_proj["y"]))

In [ ]:
dat_hdd_proj.set_crs(
            "+proj=sterea +lat_0=52.15616055555555 +lon_0=5.38763888888889 +k=0.9999079 +x_0=155000 +y_0=463000 +ellps=bessel +towgs84=565.4171,50.3319,465.5524,-0.398957388243134,0.343987817378283,-1.87740163998045,4.0725 +units=m +no_defs", 
            inplace=True)

In [ ]:
dat_hdd_proj = dat_hdd_proj.to_crs(epsg=28992)

In [ ]:
dat_hdd_proj = gpd.sjoin(dat_hdd_proj, id_wijken_23[["WK_CODE_23", "GM_CODE_23", "geometry"]], how = "left", predicate = "within")
dat_hdd_proj.head()

In [ ]:
print(sum(dat_hdd_proj.WK_CODE_23.isnull())/dat_hdd_proj.shape[0])
dat_hdd_proj = dat_hdd_proj[~dat_hdd_proj["WK_CODE_23"].isnull()]

In [ ]:
dat_hdd_g23 = (dat_hdd_proj.
               groupby(["WK_CODE_23", "GM_CODE_23"])["interpolatedObs"].
               mean().
               reset_index().
               rename(columns = {"interpolatedObs":"HDD"})
               )

## Neighborhood characteristics -- Wijktyppologie 


In [ ]:
### files and folder paths 
energy_cost = "energy_cost/Energiekosten - 2021 - Wijken (2023).csv"
energy_labels = "energy_labels/Energielabels (geldige labels Rijksoverheid) - Wijken (2023)_2020_2024.csv"
fragility_elderly = "fragility_elderly/50090NED_UntypedDataSet_18112024_154023.csv"
heating_system = "heating system/Hoofdverwarmingsinstallaties_woningen_2022_2023.xlsx"
infrastructure = "infrastructure/KEA_GroenGrijsPerBuurt_2023_03.xlsx"
renewable_energy_homes = "renewable_energy_homes/Hernieuwbare energie-installaties - Wijken (2023).csv"
energetic_efficiency = "climate_monitor/Inhoudelijke datasets per sector en thema - 2021 - Wijken.csv"
environmental_climatic = "climatic_environmental/KEA_GroenGrijsPerBuurt_2023_03_Processed.xlsx"
inequality = "income_inequality/VSO Tabel Hoofdstuk 7 2021.xlsx"
wijk_typology = "neighborhood_typology/Buurt_typologie.csv"

In [ ]:
dat_wijk_typ = pd.read_csv(os.path.join(os.getcwd(), wijk_typology), header=0, sep = ",")
dat_wijk_typ.WijktypeDef.value_counts()

In [ ]:
col_wijk_typo = {
    'Historische binnenstad':"Historic city block", 
    'Groen':"Green", 
    'Naoorlogse woonwijk':"Post_war residential",
    'Sub-urbane uitbreiding - VINEX':"Sub_urban extension_VINEX", 
    'Bloemkoolwijk':"Cauliflower_neighbourhood",
    'Vooroorlogse woonwijk':"Pre_war residential", 
    'Villa':"Villa", 
    'Tuinstad hoogbouw':"Garden city high-rise", 
    'Bedrijven':"Business",
    'Vernieuwd':"Renewed", 
    'Hoogbouw':"High-rise", 
    'Tuindorp':"Garden village", 
    'Stedelijk bouwblok':"Urban building block",
    'Volkswijk':"Working_class area", 
    'Tuinstad laagbouw':"Garden city low-rise", 
    'Buitengebied':"Rural area"
    }
list(col_wijk_typo.keys())
list(col_wijk_typo.values())

In [ ]:
dat_wijk_typ.WijktypeDef = dat_wijk_typ.WijktypeDef.replace(list(col_wijk_typo.keys()), list(col_wijk_typo.values()))
dat_wijk_typ.WijktypeDef.value_counts()

In [ ]:
sum(~gdf_buurt_2022.BU_CODE.isin(dat_wijk_typ.BU_CODE))/len(gdf_buurt_2022.BU_CODE)

In [ ]:
sum(~dat_wijk_typ.BU_CODE.isin(gdf_buurt_2022.BU_CODE))
dat_wijk_typ = dat_wijk_typ.merge(gdf_buurt_2022[["BU_CODE", "WK_CODE"]], on = "BU_CODE", how = "left")
dat_wijk_typ = dat_wijk_typ.groupby("WK_CODE").WijktypeDef.value_counts().reset_index()
dat_wijk_typ.head(n = 5)

In [ ]:
idx_pred_type = dat_wijk_typ.groupby("WK_CODE")["count"].idxmax()

In [ ]:
dat_wijk_typ = dat_wijk_typ.iloc[idx_pred_type, :]

In [ ]:
sum(~dat_wijk_typ.WK_CODE.isin(id_wijken_23.WK_CODE_23))

In [ ]:
print(dat_wijk_typ.shape)
dat_wijk_typ2 = dat_wijk_typ.merge(look_up_23_22, left_on = "WK_CODE", right_on = "WK_CODE_22", how = "left")
print(dat_wijk_typ2.shape)

In [ ]:
dat_wijk_typ2[dat_wijk_typ2.WK_CODE_23.isin(dat_wijk_typ2.WK_CODE_23[dat_wijk_typ2.WK_CODE_23.duplicated()])]

In [ ]:
idx_pred_wijk23 = dat_wijk_typ2.groupby("WK_CODE_23")["area_prop"].idxmax()
dat_wijk_typ_g23 = dat_wijk_typ2.iloc[idx_pred_wijk23, :]
dat_wijk_typ_g23 = dat_wijk_typ_g23[["WK_CODE_23", "WijktypeDef"]]
dat_wijk_typ_g23 = dat_wijk_typ_g23.rename(columns={"WijktypeDef":"wijk_predominant_typology"})
dat_wijk_typ_g23.head()

## Construction and house typology 
https://www.arcgis.com/sharing/rest/content/items/fa01ef63321e482e9b2c55620e554ffc/info/metadata/metadata.xml?format=default&output=html
The Housing Typing dataset only contains buildings that contain at least one residential object with the purpose of use “residential function”, hereinafter referred to as “homes”. These buildings are divided into the following categories in the analysis: Apartment. This is a dwelling to which multiple residential objects are related, regardless of the purpose of use of these residential objects. However, as for all properties in this dataset, at least one of these residential objects must have the purpose of use 'residential function'. Detached house. This is a dwelling that is not connected to another property with a residential object. Intermediate or semi-detached house. This dwelling is connected to multiple properties with a residential object. This category also includes semi-detached houses, these are houses that are connected by means of their garages instead of their main building. Corner house. This is the first or last house in a series of properties. This house is itself connected to a single property with a residential object, but this other property is connected to multiple other properties. Semi-detached.This property is connected to a single property with a residential object and this other property is only connected to the first property. The analysis was performed using ArcGIS tools in ArcGIS

In [ ]:
os.getcwd()

In [ ]:
dat_housing_char = pd.read_csv("BAG/main_woning_verblijf_for_R2.csv", sep = ",", header=0)

In [ ]:
dat_housing_char["house_function"] = np.where(dat_housing_char["main__pand_gebruiksdoel"].str.contains("woonfunctie"),
                                             "woonfunctie", np.NAN) 

In [ ]:
dat_housing_char = dat_housing_char[(dat_housing_char["house_function"] == "woonfunctie") & (dat_housing_char["main__pand_status"] == "Pand in gebruik")]

In [ ]:
dat_housing_char.groupby("main__pand_status").main__pand_aantal_verblijfsobjecten.sum()

In [ ]:
dat_housing_char.groupby("L0Woningtypering_Woningtypering").main__pand_aantal_verblijfsobjecten.sum()

In [ ]:
8045600 - dat_housing_char.main__pand_aantal_verblijfsobjecten.sum()

In [ ]:
dat_housing_char.main__pand_status.value_counts()

In [ ]:
dict_housing_typo = {
    'appartement':'apartment', 
    'tussenwoning/geschakeld': 'terraced_house_semi_detached', 
    'hoekwoning':'corner_house', 
    'twee-onder-een-kap': 'semi_detached',
    'vrijstaande woning':'detached'
}
dat_housing_char["t_housing_type"] = dat_housing_char["L0Woningtypering_Woningtypering"].replace(list(dict_housing_typo.keys()), list(dict_housing_typo.values()))

In [ ]:
dat_housing_char = dat_housing_char.rename(columns = {"main__pand_aantal_verblijfsobjecten":"total_residential_units"})

In [ ]:
house_year_cut = [-np.inf, 
                  1924, 
                  1947, 
                  1963, 
                  1981, 
                #   1995, 
                  2002, 
                  2015, 
                  np.inf]
dat_housing_char.main__pand_bouwjaar.value_counts()

In [ ]:
dat_housing_char["t_construction_year"] = pd.cut(dat_housing_char['main__pand_bouwjaar'], house_year_cut, labels = ["Before 1924", "1925-1947", "1948-1963", "1964-1981", "1982-2002", "2003-2015", "After 2015"], right = True)
dat_housing_char["t_construction_year"].value_counts()

In [ ]:
geometry_housing_char = dat_housing_char.geom_string.str.rsplit(" ")

In [ ]:
dat_housing_char["x"] = [ii[0] for ii in geometry_housing_char]
dat_housing_char["y"] = [ii[1] for ii in geometry_housing_char]

In [ ]:
dat_housing_char["x"] = dat_housing_char["x"].astype(float)
dat_housing_char["y"] = dat_housing_char["y"].astype(float)

In [ ]:
dat_housing_char_geo = dat_housing_char[["t_housing_type", "t_construction_year", "total_residential_units", "x", "y"]]
dat_housing_char_geo = gpd.GeoDataFrame(dat_housing_char_geo, geometry = gpd.points_from_xy(dat_housing_char_geo.x , dat_housing_char_geo.y, crs = "EPSG:28992"))
# dat_housing_char_geo = dat_housing_char_geo.to_crs("EPSG:28992")

In [ ]:
dat_housing_char_geo= dat_housing_char_geo.sjoin(id_wijken_23, predicate = "within", how = "left")
dat_housing_char_geo.head()

In [ ]:
inter_one = dat_housing_char_geo.groupby(["WK_CODE_23","t_housing_type"])["total_residential_units"].sum().reset_index()#.rename(columns = {"count":"total"})
inter_one.head()

In [ ]:
inter_one = dat_housing_char_geo.groupby(["WK_CODE_23","t_housing_type"])["total_residential_units"].sum().reset_index()#.rename(columns = {"count":"total"})
inter_one["share"] = (inter_one['total_residential_units'] / inter_one.groupby('WK_CODE_23')['total_residential_units'].transform('sum')) * 100
inter_one = (pd.pivot_table(inter_one, values = "share", index = "WK_CODE_23", columns = ["t_housing_type"]).
             reset_index().
             add_prefix('p_h_type_').
             rename(columns={"p_h_type_WK_CODE_23":"WK_CODE_23"}))
print(inter_one.shape)

inter_two = dat_housing_char_geo.groupby(["WK_CODE_23","t_construction_year",])["total_residential_units"].sum().reset_index()#.rename(columns = {"count":"total"})
inter_two["share"] = (inter_two['total_residential_units'] / inter_two.groupby('WK_CODE_23')['total_residential_units'].transform('sum')) * 100
inter_two = (pd.pivot_table(inter_two, values = "share", index = "WK_CODE_23", columns = ["t_construction_year"]).
             reset_index().
             add_prefix('p_h_year_cons_').
             rename(columns={"p_h_year_cons_WK_CODE_23":"WK_CODE_23"}))
print(inter_two.shape)


In [ ]:
inter_total = dat_housing_char_geo.groupby(["WK_CODE_23"])["total_residential_units"].sum().reset_index()#.rename(columns = {"count":"total"})
inter_total.head()

In [ ]:
dat_housing_char_g23 = inter_one.merge(inter_two, on = "WK_CODE_23")
dat_housing_char_g23 = dat_housing_char_g23.merge(inter_total, on = "WK_CODE_23" ,how = "left")
print(dat_housing_char_g23.shape)
dat_housing_char_g23.head()

# Save all intermediate tables

In [ ]:
save_intermediate = True
if (save_intermediate):
    print("Saving")
    dict_inter_tables = {
        "dat_elderly_g23" : dat_elderly_g23, 
        "dat_buurt_kaart_g23" : dat_buurt_kaart_g23,
        "dat_heating_sys_g23" : dat_heating_sys_g23,
        "dat_energetic_efficiency_g23" : dat_energetic_efficiency_g23, 
        "dat_green_gray_g23" : dat_green_gray_g23, 
        "dat_gini_g23" : dat_gini_g23,
        "dat_cdd_g23" : dat_cdd_g23,
        "dat_hdd_g23" : dat_hdd_g23, 
        "dat_wijk_typ_g23":dat_wijk_typ_g23,
        "dat_housing_char_g23":dat_housing_char_g23 
    }
    with open("00_intermediate_data/dict_intermediate_tables.pkl", "wb") as file:
        pickle.dump(dict_inter_tables, file)
else: 
    print("Loading")
    with open("00_intermediate_data/dict_intermediate_tables.pkl", "rb") as file:
        dict_inter_tables = pickle.load(file)
        dat_elderly_g23 = dict_inter_tables["dat_elderly_g23"] 
        dat_buurt_kaart_g23 = dict_inter_tables["dat_buurt_kaart_g23"] 
        dat_heating_sys_g23 = dict_inter_tables["dat_heating_sys_g23"] 
        dat_energetic_efficiency_g23 = dict_inter_tables["dat_energetic_efficiency_g23"]  
        dat_green_gray_g23 = dict_inter_tables["dat_green_gray_g23"]  
        dat_gini_g23 = dict_inter_tables["dat_gini_g23"]
        dat_cdd_g23 = dict_inter_tables["dat_cdd_g23"] 
        dat_hdd_g23 = dict_inter_tables["dat_hdd_g23"]  
        dat_wijk_typ_g23 = dict_inter_tables["dat_wijk_typ_g23"]
        dat_housing_char_g23 = dict_inter_tables["dat_housing_char_g23"]
    

# Merging all tables into 2023 geography

## Energy poverty (rental and owner occupied considered)

In [ ]:
# Energy Poverty
with open("energy_poverty/dict_ep_wijken.pkl", 'rb') as file:
    dict_ep_wijken = pickle.load(file)
ep_wijken_21 = dict_ep_wijken["21"]
ep_wijken_21.head(n = 3)

In [ ]:
ep_wijken_21 = ep_wijken_21[
    ['Regiocode', "year", 'property_type', 'total_households', 'LIHE_AndOr_LILEK',
    'LIHE_AndOr_LIZLEK', 'HEQ', 'LIHE', 'LILEK', 'LIZLEK ', 'LEKWI',
    'ZLEKWI', 'LI', 'HE', 'LEK', 'ZLEK', 'standardize_residual_income',
    'average_energy_ratio']
    ]

In [ ]:
def_wijken_23 = gdf_wijk_2023[["WK_CODE", "GM_NAAM",  "GM_CODE", "WK_NAAM", "geometry", "AANT_INW"]]
def_wijken_23 = def_wijken_23.rename(columns = {"WK_CODE":"WK_CODE_23", 
                                                "WK_NAAM":"WK_NAAM_23",
                                                "GM_CODE":"GM_CODE_23",  
                                                "GM_NAAM":"GM_NAAM_23",
                                                "AANT_INW": "AANT_INW_23"})

In [ ]:
#### Merging ###################3
# Energy Poverty 
print(def_wijken_23.shape)
def_wijken_23 = def_wijken_23.merge(ep_wijken_21, left_on= 'WK_CODE_23', right_on='Regiocode', how = 'left')
print(def_wijken_23.shape)

In [ ]:
# elderly 
print(def_wijken_23.shape)
def_wijken_23 = def_wijken_23.merge(dat_elderly_g23, on = 'WK_CODE_23', how = 'left')
print(def_wijken_23.shape)

In [ ]:
# buurtkaart 
print(def_wijken_23.shape)
def_wijken_23 = def_wijken_23.merge(dat_buurt_kaart_g23, on = 'WK_CODE_23', how = 'left')
print(def_wijken_23.shape)

In [ ]:
# heating system 
print(def_wijken_23.shape)
def_wijken_23 = def_wijken_23.merge(dat_heating_sys_g23, on = 'WK_CODE_23', how = 'left')
print(def_wijken_23.shape)

In [ ]:
# Energetic efficiency
print(def_wijken_23.shape)
def_wijken_23 = def_wijken_23.merge(dat_energetic_efficiency_g23, on = 'WK_CODE_23', how = 'left')
print(def_wijken_23.shape)

In [ ]:
# Climatic and environmental: Infrastructure 
print(def_wijken_23.shape)
def_wijken_23 = def_wijken_23.merge(dat_green_gray_g23, on = 'WK_CODE_23', how = 'left')
print(def_wijken_23.shape)


In [ ]:
# GINI
print(def_wijken_23.shape)
def_wijken_23 = def_wijken_23.merge(dat_gini_g23, on = 'WK_CODE_23', how = 'left')
print(def_wijken_23.shape)

In [ ]:
def_wijken_23["GINI"] = np.where(def_wijken_23["GINI"] == ".", np.NAN, 
                                 np.where(def_wijken_23["GINI"] == '>0,50', 0.51, def_wijken_23["GINI"]))
def_wijken_23["GINI"] = def_wijken_23["GINI"].astype(float)

In [ ]:
# Climatic HDD and CDD
print(def_wijken_23.shape)
def_wijken_23 = def_wijken_23.merge(dat_cdd_g23[["WK_CODE_23", "CDD"]], on = 'WK_CODE_23', how = 'left')
def_wijken_23 = def_wijken_23.merge(dat_hdd_g23[["WK_CODE_23", "HDD"]], on = 'WK_CODE_23', how = 'left')
print(def_wijken_23.shape)

In [ ]:
## urbanity degree 
print(def_wijken_23.shape)
def_wijken_23 = def_wijken_23.merge(gdf_wijk_2023[["STED", "WK_CODE"]], left_on = 'WK_CODE_23', right_on="WK_CODE",how = 'left')
print(def_wijken_23.shape)
def_wijken_23["STED"] = np.where(def_wijken_23["STED"] == -99999999, np.NAN, def_wijken_23["STED"])   

In [ ]:
# Wijk typology 
print(def_wijken_23.shape)
def_wijken_23 = def_wijken_23.merge(dat_wijk_typ_g23, on = 'WK_CODE_23', how = 'left')
print(def_wijken_23.shape)

In [ ]:
# housing characteristics 

print(def_wijken_23.shape)
def_wijken_23 = def_wijken_23.merge(dat_housing_char_g23, on = 'WK_CODE_23', how = 'left')
print(def_wijken_23.shape)


In [ ]:
def_wijken_23 = def_wijken_23.drop(columns = "WK_CODE")

In [ ]:
numeric_def_types = pd.DataFrame(def_wijken_23.dtypes).reset_index()
numeric_def_types = list(numeric_def_types[numeric_def_types.iloc[:, 1] == "float"]["index"])

## Calculate additional variables of interest

In [ ]:
def_wijken_23["p_low_house_energy_eff"] = def_wijken_23["p_homes_energy_label_E"] + def_wijken_23["p_homes_energy_label_F"] + def_wijken_23["p_homes_energy_label_G"]
def_wijken_23["p_medium_house_energy_eff"] = def_wijken_23["p_homes_energy_label_C"] + def_wijken_23["p_homes_energy_label_D"]
def_wijken_23["p_high_house_energy_eff"] = def_wijken_23["p_homes_energy_label_B"]
def_wijken_23["p_very_high_house_energy_eff"] = def_wijken_23["p_homes_energy_label_A_Aplus"]

In [ ]:
def_wijken_23 = def_wijken_23.rename(columns = {"fragile_health_65":"p_fragile_health_65"})

def_wijken_23["p_unemployment_rate"] = def_wijken_23["p_unemployment_rate"] * 100 
def_wijken_23["p_pension_coverage_rate"] = def_wijken_23["p_pension_coverage_rate"] * 100 
def_wijken_23["p_precarious_part_time"] = def_wijken_23["p_precarious_part_time"] * 100 

In [ ]:
def_wijken_23_toplot = def_wijken_23.copy()

In [ ]:
cols_interest = ['HEQ', 'LIHE', 'standardize_residual_income', 'average_energy_ratio', 
                 'p_fragile_health_65', 'p_with_young_children', 
                 'p_pension_coverage_rate', 'p_unemployment_rate',  'p_precarious_part_time','p_disability', 
                 'p_owner_ocupied_home', 'p_rental_home', 
                 't_gas_consum_rental', 't_gas_consum_owner', 't_elec_consum_rental', 't_elec_consum_owner', 't_gas_consum_total', 't_elec_consum_total', 
                 'p_individual_CV', 'p_block_heating', 'p_district_heat', 'p_elec_heat', 
                 'p_homes_with_solar_panels',
                 'housing_density', 'p_rental_own_hous_association',
                 'p_green', 'p_gray', 'GINI',
                 "HDD", "CDD", 'p_h_type_apartment',
                'p_h_type_corner_house', 'p_h_type_detached', 'p_h_type_semi_detached',
                'p_h_type_terraced_house_semi_detached', 'p_h_year_cons_Before 1924',
                'p_h_year_cons_1925-1947', 'p_h_year_cons_1948-1963',
                'p_h_year_cons_1964-1981', 'p_h_year_cons_1982-2002',
                'p_h_year_cons_2003-2015', 'p_h_year_cons_After 2015',
                'p_low_house_energy_eff', 'p_medium_house_energy_eff',
                'p_high_house_energy_eff', 'p_very_high_house_energy_eff'
                 ]

In [ ]:
def_wijken_23_toplot[cols_interest].describe()

In [ ]:
for column in cols_interest:
    def_wijken_23_toplot[column] = np.where(def_wijken_23_toplot[column] < 0, np.nan, def_wijken_23_toplot[column])

In [ ]:
quartiles_cols_interest = {}
for column in cols_interest:
    quartiles_cols_interest[column] = def_wijken_23_toplot[column].quantile([0.25, 0.5, 0.75])
    quartiles_cols_interest[column] = list(quartiles_cols_interest[column] )
    len_quartiles = set(quartiles_cols_interest[column])
    if len(len_quartiles) < 3:
        print(column)
        continue 
    def_wijken_23_toplot[column] = pd.qcut(def_wijken_23_toplot[column], q = 4, labels = ["Low", "Middle_Low", "Middle_High", "High"])

# Export data (csv and shp)

In [ ]:

def_wijken_23.to_csv("00_intermediate_data/def_wijken_23.csv", sep = "|", index=False)
def_wijken_23_toplot.to_csv("00_intermediate_data/def_wijken_23_quartiles.csv", sep = "|", index=False)
with open("00_intermediate_data/quartiles_cols_interest.json", "w") as file:
    json.dump(quartiles_cols_interest, file)

In [ ]:
import pyogrio
def_wijken_23.to_file("00_intermediate_data/def_wijken_23.shp") 

# Exploratory maps

In [ ]:
cols_interest2 = ['HEQ',
 'LIHE',
 'standardize_residual_income',
 'average_energy_ratio',
 'p_fragile_health_65',
 'p_with_young_children',
 'p_pension_coverage_rate',
 'p_unemployment_rate',
 'p_precarious_part_time',
 'p_disability', 
 'p_owner_ocupied_home',
 'p_rental_home',
 't_gas_consum_rental',
 't_gas_consum_owner',
 't_elec_consum_rental',
 't_elec_consum_owner',
 't_gas_consum_total',
 't_elec_consum_total',
 'p_individual_CV',
#  'p_block_heating',
#  'p_district_heat',
 'p_elec_heat',
 'p_homes_with_solar_panels',
 'p_low_house_energy_eff',
 'p_medium_house_energy_eff',
 'p_high_house_energy_eff',
 'p_very_high_house_energy_eff',
 'housing_density',
 'p_rental_own_hous_association',
 'p_green',
 'p_gray',
 'GINI',
 'HDD',
 'CDD',
'p_h_type_apartment', 
'p_h_type_corner_house', 
'p_h_type_detached', 
'p_h_type_semi_detached',
'p_h_type_terraced_house_semi_detached', 
'p_h_year_cons_Before 1924',
'p_h_year_cons_1925-1947', 
'p_h_year_cons_1948-1963',
'p_h_year_cons_1964-1981', 
'p_h_year_cons_1982-2002',
'p_h_year_cons_2003-2015', 
'p_h_year_cons_After 2015'
]

In [ ]:
# Define color scale (blue to red)
cmap = plt.cm.get_cmap("coolwarm") 
# Dynamically calculate rows and columns based on the number of plots
ncols = 2
nrows = math.ceil(len(cols_interest2) / ncols)  # Calculate rows needed
nrows

In [ ]:
fig, ax = plt.subplots(nrows=nrows, ncols=ncols, figsize=(15, 4 * nrows))
ax = ax.flatten()
for i, col in enumerate(cols_interest2):
    if col in ["p_block_heating", "p_district_heat"]:
        continue
    # if i in [0, 1]:
    def_wijken_23_toplot.plot(ax=ax[i], cmap=cmap, column=col, legend=True, 
                                legend_kwds={
        'loc': 'upper left',   # Position the legend in the upper left
        'fontsize': 7,         # Adjust font size for better readability
        'frameon': False        # Optional: Add a frame to the legend
    })
    # else:
    #     def_wijken_23_toplot.plot(ax=ax[i], cmap=cmap, column=col, legend=False)
    ax[i].set_title(col)
# plt.legend(loc='upper left')
plt.tight_layout()  # Adjust layout to prevent overlap
plt.show()